In [1]:
#from config import manifesto_key
import sys
sys.path.insert(1, '../src/')
from get_cap_dutch import Get_Cap_Data

In [2]:
a = Get_Cap_Data('../data/raw-fish/CAP/NL/','../src/cap_topic_codes.json', '../src/cap_dutch_files.json',)
cap = a.get_cap_files()
cap = cap[cap.text.notnull()]


In [ ]:
# read CMP data

def get_cmp_data(path_to_file, cmp_codes):
    
    df = pd.read_csv(path_to_file)
    df = df[['text', 'cmp_code']].dropna()
    
    for k, v in cmp_codes.items():
        if type(v) is list:
            df[k] = np.where((df['cmp_code'] == v[0] ) | (df['cmp_code'] ==v[1] ), 1, 0)
        else:
            df[k] = np.where(df['cmp_code'] == v, 1, 0)
    return df


with open('../src/cmp_topic_codes.json') as json_file: 
    cmp_codes = json.load(json_file)
    
cmp = get_cmp_data(path_to_file='../data/raw-fish/CMP/MPDS2020b_Netherlands.csv', cmp_codes = cmp_codes)


In [6]:
from traditional_sml import *

def get_classification_results(dataset, name_data):
    
    appended_data = []    
    
    for t in ['environment', 'culture', 'civil_rights', 'education', 'immigration']:
        
        print(f"starting classification of the topic ... {t}....")
        
        cr = train_classifiers(dataset, t)
    
        for i in cr:
            data = pd.DataFrame.from_dict(i)
            appended_data.append(data)
        df = pd.concat(appended_data)
        df['indicators'] = df.index

        b = get_cleaned_data(df)

        METRICS = 'f1-score'

        output_file = f'../output/tables/{name_data}_{t}_{METRICS}.html'
        print(output_file)
        b[b['metrics']==METRICS].to_html(output_file)
        print(f'saved output in {output_file}')
get_classification_results(cap, "cap")


0it [00:00, ?it/s]

starting classification of the topic ... environment....
majortopic:environment
Naive Bayes score: 0.8010244286840031
Pipeline(steps=[('vect', CountVectorizer(stop_words='english')),
                ('clf', MultinomialNB(alpha=0.01))]) are the best estimators
Linear SVM score: 0.7523781504364029
Pipeline(steps=[('vect', CountVectorizer(stop_words='english')),
                ('clf', LinearSVC(C=0.31622776601683794))]) are the best estimators
Logistic Regression score: 0.7425490196078433
Pipeline(steps=[('vect', CountVectorizer(stop_words='english')),
                ('clf', LogisticRegression(C=10.0))]) are the best estimators


1it [00:03,  3.29s/it]

Random Forest score: 0.7326732673267328
Pipeline(steps=[('vect',
                 CountVectorizer(ngram_range=(1, 2), stop_words='english')),
                ('clf', RandomForestClassifier(max_depth=2))]) are the best estimators
Naive Bayes score: 0.8213443396226416
Pipeline(steps=[('vect', TfidfVectorizer(stop_words='english')),
                ('clf', MultinomialNB(alpha=0.001))]) are the best estimators
Linear SVM score: 0.8115856651939126
Pipeline(steps=[('vect', TfidfVectorizer(stop_words='english')),
                ('clf', LinearSVC(C=0.31622776601683794))]) are the best estimators
Logistic Regression score: 0.8216248037676609
Pipeline(steps=[('vect', TfidfVectorizer(stop_words='english')),
                ('clf', LogisticRegression(C=10.0))]) are the best estimators


2it [00:06,  3.03s/it]
0it [00:00, ?it/s]

...

Random Forest score: 0.8249661933739014
Pipeline(steps=[('vect', TfidfVectorizer(stop_words='english')),
                ('clf', RandomForestClassifier(max_depth=2))]) are the best estimators
..........loaded the data frame........
../output/tables/cap_education_f1-score.html
saved output in ../output/tables/cap_education_f1-score.html
starting classification of the topic ... immigration....
majortopic:immigration
Naive Bayes score: 0.8740229885057471
Pipeline(steps=[('vect', CountVectorizer(stop_words='english')),
                ('clf', MultinomialNB(alpha=0.01))]) are the best estimators
Linear SVM score: 0.7767275097783573
Pipeline(steps=[('vect', CountVectorizer(stop_words='english')),
                ('clf', LinearSVC(C=0.01))]) are the best estimators
Logistic Regression score: 0.7907027498908774
Pipeline(steps=[('vect', CountVectorizer(stop_words='english')),
                ('clf', LogisticRegression(C=0.31622776601683794))]) are the best estimators


1it [00:03,  3.81s/it]

Random Forest score: 0.5318562610229277
Pipeline(steps=[('vect', CountVectorizer(stop_words='english')),
                ('clf', RandomForestClassifier(max_depth=2))]) are the best estimators
Naive Bayes score: 0.8803075310152019
Pipeline(steps=[('vect',
                 TfidfVectorizer(ngram_range=(1, 2), stop_words='english')),
                ('clf', MultinomialNB(alpha=0.01))]) are the best estimators
Linear SVM score: 0.8633037694013304
Pipeline(steps=[('vect',
                 TfidfVectorizer(ngram_range=(1, 2), stop_words='english')),
                ('clf', LinearSVC(C=10.0))]) are the best estimators
Logistic Regression score: 0.8705031971087018
Pipeline(steps=[('vect',
                 TfidfVectorizer(ngram_range=(1, 2), stop_words='english')),
                ('clf', LogisticRegression(C=10.0))]) are the best estimators


2it [00:07,  3.84s/it]

Random Forest score: 0.513968957871397
Pipeline(steps=[('vect', TfidfVectorizer(stop_words='english')),
                ('clf', RandomForestClassifier(max_depth=2))]) are the best estimators
..........loaded the data frame........
../output/tables/cap_immigration_f1-score.html
saved output in ../output/tables/cap_immigration_f1-score.html


In [7]:
get_classification_results(cmp, "cmp")


0it [00:00, ?it/s]

starting classification of the topic ... environment....
majortopic:environment
Naive Bayes score: 0.8293316831683168
Pipeline(steps=[('vect', CountVectorizer(stop_words='english')),
                ('clf', MultinomialNB(alpha=0.01))]) are the best estimators
Linear SVM score: 0.8179340133193709
Pipeline(steps=[('vect', CountVectorizer(stop_words='english')),
                ('clf', LinearSVC(C=0.31622776601683794))]) are the best estimators
Logistic Regression score: 0.81982401191225
Pipeline(steps=[('vect', CountVectorizer(stop_words='english')),
                ('clf', LogisticRegression(C=0.31622776601683794))]) are the best estimators


1it [00:24, 24.30s/it]

Random Forest score: 0.6944426411944727
Pipeline(steps=[('vect', CountVectorizer(stop_words='english')),
                ('clf', RandomForestClassifier(max_depth=2))]) are the best estimators
Naive Bayes score: 0.8282670930802363
Pipeline(steps=[('vect', TfidfVectorizer(stop_words='english')),
                ('clf', MultinomialNB(alpha=0.01))]) are the best estimators
Linear SVM score: 0.8263939509329272
Pipeline(steps=[('vect', TfidfVectorizer(stop_words='english')),
                ('clf', LinearSVC(C=0.31622776601683794))]) are the best estimators


/Users/anne/Library/Python/3.8/lib/python/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...

Random Forest score: 0.7233141894508938
Pipeline(steps=[('vect', TfidfVectorizer(stop_words='english')),
                ('clf', RandomForestClassifier(max_depth=2))]) are the best estimators
..........loaded the data frame........
../output/tables/cmp_education_f1-score.html
saved output in ../output/tables/cmp_education_f1-score.html
starting classification of the topic ... immigration....
majortopic:immigration
Naive Bayes score: 0.7245026824289038
Pipeline(steps=[('vect',
                 CountVectorizer(ngram_range=(1, 2), stop_words='english')),
                ('clf', MultinomialNB(alpha=0.01))]) are the best estimators
Linear SVM score: 0.7423312883435583
Pipeline(steps=[('vect',
                 CountVectorizer(ngram_range=(1, 2), stop_words='english')),
                ('clf', LinearSVC(C=0.31622776601683794))]) are the best estimators
Logistic Regression score: 0.7484662576687117
Pipeline(steps=[('vect',
                 CountVectorizer(ngram_range=(1, 2), stop_words='englis

1it [00:02,  2.86s/it]

Random Forest score: 0.6545525902668761
Pipeline(steps=[('vect', CountVectorizer(stop_words='english')),
                ('clf', RandomForestClassifier(max_depth=2))]) are the best estimators
Naive Bayes score: 0.7241674351393232
Pipeline(steps=[('vect',
                 TfidfVectorizer(ngram_range=(1, 2), stop_words='english')),
                ('clf', MultinomialNB(alpha=0.01))]) are the best estimators
Linear SVM score: 0.7447045392359439
Pipeline(steps=[('vect',
                 TfidfVectorizer(ngram_range=(1, 2), stop_words='english')),
                ('clf', LinearSVC(C=0.31622776601683794))]) are the best estimators
Logistic Regression score: 0.7146997656845492
Pipeline(steps=[('vect',
                 TfidfVectorizer(ngram_range=(1, 2), stop_words='english')),
                ('clf', LogisticRegression(C=0.31622776601683794))]) are the best estimators


2it [00:05,  2.70s/it]

Random Forest score: 0.6286169970380495
Pipeline(steps=[('vect', TfidfVectorizer(stop_words='english')),
                ('clf', RandomForestClassifier(max_depth=2))]) are the best estimators
..........loaded the data frame........
../output/tables/cmp_immigration_f1-score.html
saved output in ../output/tables/cmp_immigration_f1-score.html


In [ ]:
import json

topics = { "environment": "501", "culture" : "502" , "civil_rights" : "201", "education" : ["506", "507"], 
                "immigration": ["601", "602"] }

with open('../src/cmp_topic_codes.json', 'w') as fp:
    json.dump(topics, fp)